In [12]:
!pip install gym
!pip install torch
!pip install matplotlib
!pip install streamlit
!pip install pyngrok

In [13]:
%%writefile app.py
import streamlit as st
import numpy as np
import torch
import torch.nn as nn

# ========== MODEL AND ENV SETUP ==========

class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.fc(x)

class CabEnv:
    def __init__(self):
        self.hours = 24
        self.locations = 5
        self.destinations = 5
        self.reset()

    def reset(self):
        self.time = np.random.randint(0, self.hours)
        self.location = np.random.randint(0, self.locations)
        return self._get_state()

    def _get_state(self):
        return np.array([self.time, self.location], dtype=np.float32)

    def step(self, action):
        ride_distance = np.random.randint(1, 10)
        reward = ride_distance * 10 - 5
        self.time = (self.time + ride_distance) % 24
        self.location = action
        return self._get_state(), reward

# ========== STREAMLIT APP ==========

st.set_page_config(page_title="Cab Ride DQN", layout="centered")
st.title("🚕 Cab Ride Recommendation using DQN")

if "env" not in st.session_state:
    st.session_state.env = CabEnv()
    st.session_state.model = DQN(2, 5)
    st.session_state.model.load_state_dict(torch.load("dqn_model.pth", map_location=torch.device('cpu')))
    st.session_state.total_reward = 0
    st.session_state.state = st.session_state.env.reset()

state = st.session_state.state
model = st.session_state.model
env = st.session_state.env

st.subheader("Current State:")
st.write(f"🕒 Time: {int(state[0])}, 📍 Location: {int(state[1])}")

with torch.no_grad():
    state_tensor = torch.FloatTensor(state)
    q_values = model(state_tensor)
    action = torch.argmax(q_values).item()

st.success(f"Recommended Ride (Destination): {action}")

if st.button("🚀 Take Ride"):
    next_state, reward = env.step(action)
    st.session_state.total_reward += reward
    st.session_state.state = next_state

    st.write(f"💰 Reward from ride: {reward:.2f}")
    st.write(f"📈 Total Earnings: {st.session_state.total_reward:.2f}")
    st.rerun()


Overwriting app.py


In [14]:
torch.save(model.state_dict(), "dqn_model.pth")


In [11]:
!ngrok config add-authtoken 2v0EBSyxkDB56hpJj6hYJRaVU7G_KaWbWH7gJpL757bMTc3p


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
import threading
import time
from pyngrok import ngrok

# Start streamlit in a separate thread
def run_app():
    !streamlit run app.py

threading.Thread(target=run_app).start()

# Wait for Streamlit to boot
time.sleep(5)

# Setup public tunnel manually with correct syntax
public_url = ngrok.connect(addr="8501", proto="http")
print("🔗 Public Streamlit URL:", public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8504
  Network URL: http://172.28.0.12:8504
  External URL: http://34.82.184.114:8504

🔗 Public Streamlit URL: NgrokTunnel: "https://c658-34-82-184-114.ngrok-free.app" -> "http://localhost:8501"
